# Mentions of competitors and their activity on Twitter

In [2]:
from datetime import datetime, timedelta
import json
import nest_asyncio
import pandas as pd
import requests
import twint

nest_asyncio.apply()

## Twint

In [51]:
def twint_search(query, start, end, query_type):
    c = twint.Config()
    if query_type == "search":
        c.Search = query
    elif query_type == "username":
        c.Username = query
    else:
        raise BaseException("wrong type")
    c.Since = start
    c.Until = end
    c.Pandas = True
    twint.run.Search(c)
    return twint.storage.panda.Tweets_df

In [52]:
def twint_to_data():
    with open("./queries_list.json", "r") as file:
        competitors_queries = json.loads(file)

    start_datetime = datetime.now() - timedelta(hours=1)
    start = "{date} {hour}:00:00".format(
        date=str(start_datetime).split()[0], hour=str(start_datetime.hour)
    )
    end_date = datetime.now()
    end = "{date} {hour}:00:00".format(
        date=str(end_date).split()[0], hour=str(end_date.hour)
    )

    data = pd.DataFrame()
    competitors = []

    for competitor in competitors_queries:
        query = competitors_queries[competitor]["search_keyword"]
        print(query)
        query_type = "search"

        tweets = twint_search(query=query, start=start, end=end, query_type=query_type)
        data = pd.concat([data, tweets], ignore_index=True)
        competitors += [competitor] * len(tweets)

        if "usernames" in competitors_queries[competitor]:
            username = competitors_queries[competitor]["usernames"]["Twitter"]
            query_type = "username"
            tweets = twint_search(
                query=query, start=start, end=end, query_type=query_type
            )
            data = pd.concat([data, tweets], ignore_index=True)
            competitors += [competitor] * len(tweets)

    data["competitor"] = competitors

    return data

## Coda

In [53]:
def twitter_to_coda():
    data = twint_to_data()
    headers = {"Authorization": coda.TOKEN}

    uri = f"https://coda.io/apis/v1/docs/{coda.DOC}/tables/{coda.TABLE}/rows"

    columns = {
        "Competitor": "coda.COL_1",
        "Social media": "coda.COL_2",
        "Subreddit/hashtags": "coda.COL_3",
        "Content": "coda.COL_4",
        "Author": "coda.COL_5",
        "Metrics": "coda.COL_6",
        "Link": "coda.COL_7",
        "Date": "coda.COL_8",
    }

    rows = [
        {
            "cells": [
                {"column": columns["Competitor"], "value": data["competitor"][row]},
                {"column": columns["Social media"], "value": "Twitter"},
                {
                    "column": columns["Subreddit/hashtags"],
                    "value": ", ".join(data["hashtags"][row]),
                },
                {"column": columns["Content"], "value": data["tweet"][row]},
                {"column": columns["Author"], "value": data["username"][row]},
                {
                    "column": columns["Metrics"],
                    "value": "Likes: {likes},\nReplies: {replies}".format(
                        likes=data["nlikes"][row],
                        replies=data["nreplies"][row],
                        retweets=data["nretweets"][row],
                    ),
                },
                {"column": columns["Link"], "value": data["link"][row]},
                {"column": columns["Date"], "value": data["date"][row]},
            ]
        }
        for row in range(len(data))
    ]

    payload = {"rows": rows}

    req = requests.post(uri, headers=headers, json=payload)
    req.raise_for_status()
    res = req.json()
    res

    return